In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader,Dataset
import torch.nn.functional as F
import torch.nn as nn

In [2]:
df=pd.read_csv(r"C:\Users\ASUS\OneDrive\Desktop\code\datasets\drug200.csv")
df.head()

,Age,Sex,BP,Cholesterol,Na_to_K,Drug
0,23,F,HIGH,HIGH,25.355,DrugY
1,47,M,LOW,HIGH,13.093,drugC
2,47,M,LOW,HIGH,10.114,drugC
3,28,F,NORMAL,HIGH,7.798,drugX
4,61,F,LOW,HIGH,18.043,DrugY


In [3]:
df[df.isna().any(axis=1)]

,Age,Sex,BP,Cholesterol,Na_to_K,Drug


In [4]:
df.describe(include='all')

,Age,Sex,BP,Cholesterol,Na_to_K,Drug
count,200.000000,200,200,200,200.000000,200
unique,NaN,2,3,2,NaN,5
top,NaN,M,HIGH,HIGH,NaN,DrugY
freq,NaN,104,77,103,NaN,91
mean,44.315000,NaN,NaN,NaN,16.084485,NaN
std,16.544315,NaN,NaN,NaN,7.223956,NaN
min,15.000000,NaN,NaN,NaN,6.269000,NaN
25%,31.000000,NaN,NaN,NaN,10.445500,NaN
50%,45.000000,NaN,NaN,NaN,13.936500,NaN
75%,58.000000,NaN,NaN,NaN,19.380000,NaN


In [5]:
a=set(df['Drug'])
b=set(df['BP'])
c=set(df['Cholesterol'])
print(a,b,c)

{'drugC', 'drugA', 'DrugY', 'drugX', 'drugB'} {'LOW', 'HIGH', 'NORMAL'} {'HIGH', 'NORMAL'}


In [6]:
from pandas import get_dummies
dfv2=get_dummies(df,columns=['Sex','BP','Cholesterol','Drug'],dtype="float")


In [7]:
dfv2.head()

,Age,Na_to_K,Sex_F,Sex_M,BP_HIGH,BP_LOW,BP_NORMAL,Cholesterol_HIGH,Cholesterol_NORMAL,Drug_DrugY,Drug_drugA,Drug_drugB,Drug_drugC,Drug_drugX
0,23,25.355,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
1,47,13.093,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,47,10.114,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,28,7.798,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
4,61,18.043,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0


In [8]:
col=list(dfv2.columns)
ycol=[]
[ycol.append(x) for x in col if x.startswith('Drug')]
[col.remove(x) for x in ycol]

[None, None, None, None, None]

In [9]:
X=dfv2[col]
Y=dfv2[ycol]

xarray=np.array(X)
yarray=np.array(Y)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(xarray, yarray, test_size=0.2, random_state=42)
X_val,X_test,y_val,y_test=train_test_split(X_test,y_test, test_size=0.5,random_state=42)

In [11]:
print(X_train.shape,y_train.shape)

(160, 9) (160, 5)


In [12]:
class dataset(Dataset):
    def __init__(self,A,B):
        self.X=torch.tensor(A,dtype=torch.float32)
        self.Y=torch.tensor(B,dtype=torch.float32)
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, index):
        return self.X[index],self.Y[index]

In [13]:
traindataset=dataset(A=X_train,B=y_train)
valdataset=dataset(A=X_val,B=y_val)
testdataset=dataset(A=X_test,B=y_test)

traindata_laoder= DataLoader(traindataset, batch_size= 10, shuffle=True)
testdata_loader= DataLoader(testdataset, batch_size= 10,shuffle=True)
valdata_loader=DataLoader(valdataset,batch_size= 10,shuffle=True)

In [26]:
class Classification(nn.Module):
    def __init__(self):
        super(Classification,self).__init__()
        self.Linear1=nn.Linear(X.shape[1],10)
        self.Linear2=nn.Linear(10,Y.shape[1])
            
    def forward(self,x):
        x=self.Linear1(x)
        x=self.Linear2(x)
        return x
model=Classification()

In [27]:
from torchsummary import summary

summary(model,input_size=(xarray.shape[1],))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 10]             100
            Linear-2                    [-1, 5]              55
Total params: 155
Trainable params: 155
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------


In [28]:
import torch.optim as optim
critetion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=1e-3)

In [29]:

for i in range(120):
    ttloss=0
    tacc=0
    ttsample=0

    vtloss=0
    vacc=0
    vtsample=0

    model.train()

    for value,label in traindata_laoder:
        optimizer.zero_grad()
        output=model(value)
        if label.dim() > 1:
            label = torch.argmax(label, dim=1)
        # loss
        batchloss=critetion(output,label)
        ttloss+=batchloss.item()*value.size(0)
        # accuracy
        _, preds = torch.max(output, 1)
        tacc+=(preds == label).sum().item()
        ttsample += label.size(0)

        batchloss.backward()
        optimizer.step()
        # optimizer.zero_grad()

    model.eval()
    with torch.no_grad():
        for value,label in valdata_loader:
            if label.dim() > 1:
                label = torch.argmax(label, dim=1)
            output=model(value)
            # loss
            batchloss=critetion(output,label)
            vtloss+=batchloss.item()*value.size(0)
            # accuracy
           
            _, preds = torch.max(output, 1)
            vacc += (preds == label).sum().item()
            vtsample += label.size(0)

    tatl = ttloss / len(traindata_laoder.dataset)
    ta = (tacc / ttsample)*100 

    vatl = vtloss / len(valdata_loader.dataset)
    va= (vacc / vtsample)*100

    print(f"""Epoch:{i+1}
          
          train_loss:{tatl} ,train_accuracy{ta}
          val_loss: {vatl} ,val_accuracy: {va}""")
    
    print("="*20)           


Epoch:1
          
          train_loss:3.048769064247608 ,train_accuracy47.5
          val_loss: 2.9203635454177856 ,val_accuracy: 35.0
Epoch:2
          
          train_loss:1.9143274277448654 ,train_accuracy48.75
          val_loss: 1.7374166250228882 ,val_accuracy: 35.0
Epoch:3
          
          train_loss:1.1934584453701973 ,train_accuracy53.125
          val_loss: 1.1715489625930786 ,val_accuracy: 50.0
Epoch:4
          
          train_loss:1.0231591165065765 ,train_accuracy61.25000000000001
          val_loss: 1.1581799387931824 ,val_accuracy: 50.0
Epoch:5
          
          train_loss:1.0053367912769318 ,train_accuracy60.0
          val_loss: 1.1746883988380432 ,val_accuracy: 50.0
Epoch:6
          
          train_loss:0.9850547760725021 ,train_accuracy60.62499999999999
          val_loss: 1.1359978020191193 ,val_accuracy: 50.0
Epoch:7
          
          train_loss:0.969492930918932 ,train_accuracy60.0
          val_loss: 1.1277180314064026 ,val_accuracy: 50.0
Epoch:8

In [30]:
teloss=0
tesample=0
teacc=0
with torch.no_grad():
        for value,label in testdata_loader:
            if label.dim() > 1:
                label = torch.argmax(label, dim=1)
            output=model(value)
            # loss
            batchloss=critetion(output,label)
            teloss+=batchloss.item()*value.size(0)
            # accuracy
           
            _, preds = torch.max(output, 1)
            teacc += (preds == label).sum().item()
            tesample += label.size(0)

        tetl= average_total_loss = teloss / len(testdata_loader.dataset)
        tea= (teacc / tesample)*100

        print(f"test_loss:{tetl} ,test_accuracy{tea}")
        

test_loss:0.24349480867385864 ,test_accuracy95.0
